# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Regression because output is discrete. 


## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split

In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
#explore data
n_students = len(student_data.axes[0])
n_features = len(student_data.axes[1]) - 1 #exclude last feature
n_passed =  len(student_data[student_data.passed=='yes'].axes[0])
n_failed = len(student_data[student_data.passed=='no'].axes[0])
grad_rate = (n_passed * 100.0) / n_students
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [38]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [39]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)

print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [46]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
#select the most "important" features (subjective)
X_all = X_all[['Pstatus_A', 'Pstatus_T', 'Medu','Fedu','studytime','failures','higher','famrel','Dalc','Walc','health','absences']]

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, test_size=num_test, random_state=1)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])

#print X_train.head()
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.


For this assignment the following supervised learning models are chosen:  
- Support vector machine (SVM) linear classifier
    
- Decision Tree classifier
- (Ensemble) Gradient Boosting Classifier



In [41]:
# Train a model
import time
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

svm = svm.SVC()
tree = DecisionTreeClassifier()
boost = GradientBoostingClassifier()

# Fit model to training data
train_classifier(svm, X_train, y_train)  # note: using entire training set here
train_classifier(tree, X_train, y_train)
train_classifier(boost, X_train, y_train)

#print clf
#print tree
#print boost
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.005
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Training GradientBoostingClassifier...
Done!
Training time (secs): 0.102


In [42]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score_svm = predict_labels(svm, X_train, y_train)
train_f1_score_tree = predict_labels(tree, X_train, y_train)
train_f1_score_boost = predict_labels(boost, X_train, y_train)

print "F1 score for training set (SVM): {}".format(train_f1_score_svm)
print "F1 score for training set (Tree): {}".format(train_f1_score_tree)
print "F1 score for training set (Boost): {}".format(train_f1_score_boost)


Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set (SVM): 0.861538461538
F1 score for training set (Tree): 0.99746835443
F1 score for training set (Boost): 0.906542056075


In [43]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(svm, X_test, y_test))
print "F1 score for test set: {}".format(predict_labels(tree, X_test, y_test))
print "F1 score for test set: {}".format(predict_labels(boost, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.847682119205
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.672
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.851063829787


In [44]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# Note: Keep the test set constant
for experiment_train_size in [100, 200, 300]:
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=experiment_train_size, test_size=num_test, random_state=1)

    train_predict(svm, X_train, y_train, X_test, y_test)
    train_predict(tree, X_train, y_train, X_test, y_test)
    train_predict(boost, X_train, y_train, X_test, y_test)



------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.001
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.855263157895
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.815789473684
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.651162790698
------------------------------------------
Training set size: 100
Training GradientBoostingClassifier...
Done!
Training time (secs): 0.081
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.984848484848
Predicting labels using

In [ ]:
# TODO: Train and predict using two other models

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?


Choose Gradient Boosting


In [45]:
# TODO: Fine-tune your model and report the best F1 score
#choose gradient boosting 
#parameters of gradient boosting to optimize: learning_rate, n_estimators, max_depth
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

def performance_metric(y_true, y_predict):
    return f1_score(y_true, y_predict, pos_label='yes')


def find_optimize_boost_parameters():
    
    start = time.time()


    #params = {'n_estimators':(50, 100, 200, 500, 1000),
    #          'learning_rate':(0.01, 0.02, 0.04, 0.08, 0.1, 0.2, 0.4, 0.8 )}
    # -> return n_estimators:200, learning_rate:0.01

    params = {'n_estimators':(50, 100, 200, 500, 1000),
              'learning_rate':(0.01, 0.02, 0.1),
              'max_depth':(1, 2, 3, 4),
              'loss':('deviance', 'exponential')
             }

    scorer = make_scorer(performance_metric, greater_is_better = True)
    grid_search = GridSearchCV(boost, params, scoring = scorer)
    #print grid_search 
    grid_search.fit(X_all, y_all)

    end = time.time()

    print "Done!\n GridSearch CV time (secs): {:.3f}".format(end - start)

    return grid_search.best_params_

best_params = find_optimize_boost_parameters()
print best_params

boost = GradientBoostingClassifier(n_estimators = best_params['n_estimators'], 
                                   learning_rate = best_params['learning_rate'], 
                                   max_depth = best_params['max_depth'], 
                                   loss = best_params['loss'])

for experiment_train_size in [100, 200, 300]:
    X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=experiment_train_size, test_size=num_test, random_state=1)
    train_predict(boost, X_train, y_train, X_test, y_test)




Done!
 GridSearch CV time (secs): 106.407
{'n_estimators': 100, 'loss': 'deviance', 'learning_rate': 0.01, 'max_depth': 1}
------------------------------------------
Training set size: 100
Training GradientBoostingClassifier...
Done!
Training time (secs): 0.039
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.807453416149
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.838709677419
------------------------------------------
Training set size: 200
Training GradientBoostingClassifier...
Done!
Training time (secs): 0.039
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.815950920245
Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.8375
------------------------------------------
Training set size: 300
Training GradientBoos